# <span style='background-color:#fff5b1'>Down sampling과 Up sampling
- Down sampling
    - Convolution을 진행하면서 입력 이미지의 크기를 줄여가는 것
    - convolution의 stride나 Pooling layer를 이용해 줄인다.
- Up sampling
    - Convolution을 진행하면서 입력 이미지의 크기를 늘려가는 것
    - 보통 convolution의 stride나 Pooling layer를 이용해 줄여진 이미지를 다시 원래 크기로 복원 시킬때 많이 사용된다.

# <span style='background-color:#fff5b1'> UpSampling
- tensorflow.keras.layers.UpSampling2D 사용.
- 단순히 늘린다. 
    - 크기를 늘리는 resizing을 한 뒤 빈 공간을 채운다.
- UpSampling2D로 입력 이미지의 사이즈를 늘린 뒤 Conv2D를 연결해서 학습이 되도록 한다.
- 하이퍼파라미터
    - size=(2,2) : 입력을 몇배로 크게 만들지 지정

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
X = np.arange(1, 5).reshape(1, 2, 2, 1)
X.shape

In [ ]:
# interpolation='nearest' : defautl, 'bilinear' 두가지 방식 제공
model = keras.Sequential()
model.add(layers.UpSampling2D(size=2, input_shape=(2,2,1)))
model.summary()

In [ ]:
X_up = model.predict(X)
X_up.shape, X.shape

In [ ]:
X_up.reshape(10,10)

##  <span style='background-color:#fff5b1'>Transpose Convolution
- tensorflow.keras.layers.Conv2DTranspose 를 이용
- parameter가 있는 Filter를 이용해 입력 이미지의 사이즈를 늘린다.
    - convolution 계산을 역으로 하는 방식을 사용한다.
- padding을 same으로 하고 strides로 크기를 정한다.
    - size가 strides로 지정한 배수 만큼 늘어난다.


In [ ]:
model = keras.Sequential()
model.add(layers.Conv2DTranspose(kernel_size=3, filters=12, 
                                 strides=2, padding='same', input_shape=(2,2,1)))

In [ ]:
X_up2 = model.predict(X)
X.shape, X_up2.shape

# <span style='background-color:#fff5b1'>DCGAN 

- Deep Convolutional Network + GAN
- 처음 GAN 모델은 Dense Layer를 사용했는데 이것을 Convolution Layer로 변경함.

## <span style='background-color:#fff5b1'>DCGAN의 안정적 학습을 위한 가이드라인</span>
1. Convolution 레이어에<span style='background-color:#fff5b1'>**Pooling Layer를 사용하지 않는다.**</span>
2. 안정적 학습을 위해 <span style='background-color:#fff5b1'>**BatchNormalization 사용.**</span>
3. Fully Connected Layer (Dense) 를 사용하지 않는다. (Discriminator의 출력은 예외)
4. Generator의 Hidden Layer에는  <span style='background-color:#fff5b1'>**LeakyReLU activation**</span>을 사용하고 **출력 Layer는 Tanh를 사용.**</span>
5. Discriminator의  <span style='background-color:#fff5b1'>**모든 Layer는 LeakyReLU activation을 사용한다.**</span> (Discriminator의 출력은 예외)

In [ ]:
img_shape = (28, 28, 1)
z_dim = 100 

# 생성자
- 이미지 upsampling
    - 7X7 => 14X14 => 28X28 로 키운다.
    - Transpose Convolution 사용
        - Conv2DTranspose

In [ ]:
def create_generator(z_dim=100):
    model = keras.Sequential()
    
    model.add(layers.Dense(7*7*256, input_shape=(z_dim, )))
    model.add(layers.Reshape((7, 7, 256)))

    # Convolution : Conv2DTranspose -> BatchNormalization -> Activation(LeakyReLU)
    model.add(layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))

    model.add(layers.Conv2DTranspose(filters=64, kernel_size=3, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))

    model.add(layers.Conv2DTranspose(filters=1, kernel_size=3, strides=2, padding='same'))
    model.add(layers.Activation("tanh"))

    return model

# 판별자
- 28 X 28 input => 14 X 14 => 7 X 7 => 3 X 3으로 절반씩 down sampling

In [ ]:
def create_discriminator(img_shape):
    model = keras.Sequential()
    
    model.add(layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same',
                            input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.01))

    model.add(layers.Conv2D(filters=64, kernel_size=3, strides=2,padding='same'))
    model.add(layers.LeakyReLU(alpha=0.01))

    model.add(layers.Conv2D(filters=128, kernel_size=3, strides=2,padding='same'))
    model.add(layers.LeakyReLU(alpha=0.01))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# GAN 모델
- 생성자 + 판별자

In [ ]:
def create_gan(generator, discriminator):
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)

    return model

# 모델 생성 및 컴파일

In [ ]:
discriminator = create_discriminator(img_shape)
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

generator = create_generator(z_dim)
discriminator.trainable=False

gan = create_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# 훈련

In [ ]:
import matplotlib.pyplot as plt
def sample_images(generator, image_grid_row=4, image_grid_col=4):
    """
    Generator를 이용해 가짜 이미지를 생성해 그리는 함수.
    그리드 행, 열의 개수를 받아 행 * 열 개수만큼 그린다.
    [매개변수]
        generator: Generator 모델
        image_grid_rows: 이미지를 그릴 grid 행수 (기본값 : 4)
        image_grid_columns: 이미지를 그릴 grid 열수(기본값 : 4)
    """
    z = np.random.normal(0, 1, (image_grid_row*image_grid_col, z_dim))
    gen_images = generator.predict(z)
    plt.figure(figsize=(7,7))
    for i in range(image_grid_row * image_grid_col):
        plt.subplot(image_grid_row, image_grid_col, i+1)
        plt.imshow(gen_images[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.show()

In [ ]:
sample_images(generator)

## training 함수

In [ ]:
loss_list = []
acc_list = []
iteration_list = []
def train(train_image, iterations, batch_size, sample_interval):
    train_image = train_image/127.5-1 # -1 ~ 1사이로 scaling
    train_image = train_image[..., np.newaxis] #채널 차원 증가

    # Label 생성
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    # 학습.
    for iteration in range(iterations):
        #판별자 학습
        # 정답에서 추출할 이미지의 index를 random 함수를 이용해 batch 개수만큼 조회
        idx = np.random.randint(0, train_image.shape[0], batch_size)
        # 학습에 사용할 정답 이미지들 조회
        real_imgs = train_image[idx]
        
        # Fake image를 만들기 위해 generator에 넣어줄 잡음 생성.
        z = np.random.normal(0,1, (batch_size, 100))
        # Generator를 이용해 Fake image 생성
        gen_imgs = generator.predict(z)
        
        ##################학습: (한번에 다 학습 시키는 것이 아니라 조금씩 판별자와 생성자를 같이 학습시켜야 하기 때문에 train_on_batch()사용)
        # 1. 진짜 이미지로 discriminator(판별자)를 배치만큼 학습시킨다. 
        # 2. generator(생성자)가 생성한 가짜 이미지로 discriminator(판별자)를 학습시킨다.
        # 3. 1과 2의 loss 평균이 판별자의 로스이다.
        #
        # 4. generator(생성자) 학습 ---> gan을 이용해 학습시킨다.
        #    생성자가 만든 이미지를 판별자가 진짜라고 판정해야 하므로 input으로 잡음과 정답을 전달한다.
        #    그래서 loss가 작으면 생성자가 찾은 것을 판별자가 정답으로 생각한 것이므로 생성자가 잘 만들어진것이 된다.
        #
        #######################################################################
        #진짜 이미지로 학습
        d_loss_acc_real = discriminator.train_on_batch(real_imgs, real)
        #생성자가 만든 가짜 이미지로 학습
        d_loss_acc_fake = discriminator.train_on_batch(gen_imgs, fake)

        d_loss, acc = np.add(d_loss_acc_real, d_loss_acc_fake)*0.5 # * 0.5는 두 loss 평균계산을 위해.

        ## 생성자 훈련 - gan을 이용해서 훈련. 
        z = np.random.normal(0, 1, (batch_size, 100))
        gan_loss = gan.train_on_batch(z, real) # input으로 잡음과 정답을 전달

        # 중간결과 확인
        if iteration % sample_interval == 0:
            loss_list.append((d_loss, gan_loss))
            acc_list.append(acc)
            iteration_list.append(iteration)
            print(f'판별자 손실:{d_loss}, 판별자정확도:{acc}, gan 손실:{gan_loss}')
            sample_images(generator)

In [ ]:
import time
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
iterations = 5000
batch_size=100
sample_interval=1000
start = time.time()
train(X_train, iterations, batch_size, sample_interval)
end = time.time()
print((end-start)/60, '분')

# 이미지 생성

In [ ]:
z = np.random.normal(0, 1, (3, 100)) 
pred = generator.predict(z)

In [ ]:
plt.figure(figsize=(10,5))
for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.imshow(pred[i].reshape(28,28), cmap='gray')
    plt.axis('off')
plt.show()    